In [1]:
import math
import random

import numpy as np
from tqdm import tqdm

from utils import *
from Reed_Solomon import *
from Convolutional import *

### Encoder/Decoders

Create Reed-Solomon Encoder

In [2]:
m = 8
t = 16
# default mode is np array
RS_255_223 = ReedSolomon(m, t)

In [3]:
n = RS_255_223.n
k = RS_255_223.k

n, k

(255, 223)

Create Convolutional Encoder

In [5]:
kern_7_2 = np.array([binary_str_to_array('10101011'), binary_str_to_array('10000101')])
conv = ConvCode(kern_7_2)

### Utility functions
the error is modelled by BSC
the coding scheme is created as concatenation between (255, 223) R-S code and (7, 1/2) convolution code

In [6]:
def simulate_error(ay, p):
    l = ay.shape[0]
    errs = np.random.binomial(1, p, (l,))
    return (ay + errs) % 2

In [7]:
def voyager_encode(msg):
    return conv.encode(RS_255_223.encode(msg))

def voyager_decode(code):
    err_corrected, recovered = RS_255_223.decode(conv.decode(code))
    return recovered

### Initial Trial
Try encoding and decoding using both coding systems

In [8]:
message_length = m * k

In [17]:
msg = generate_binary_array(message_length)

In [18]:
encoded = voyager_encode(msg)

In [19]:
received = simulate_error(encoded, 0.005)

In [20]:
# the hamming distance, how many errors are there?
hamming_dist(encoded, received)

23

In [21]:
recovered = voyager_decode(received)

In [22]:
np.sum(recovered == msg)

1784

In [25]:
np.all(recovered == msg)

True

### Actual experiment

In [67]:
# number of trials
num_trials = 100

# min and max channel error
min_p = 0.10
max_p = 0.15

# number of intervals
num_interval = 20

p_values = np.linspace(min_p, max_p, num_interval+1)

In [68]:
p_values

array([0.1   , 0.1025, 0.105 , 0.1075, 0.11  , 0.1125, 0.115 , 0.1175,
       0.12  , 0.1225, 0.125 , 0.1275, 0.13  , 0.1325, 0.135 , 0.1375,
       0.14  , 0.1425, 0.145 , 0.1475, 0.15  ])

In [72]:
num_errors_list_list = []
num_failure_list = []

In [73]:
for p in tqdm(p_values[1:]):
    num_errors_list = []
    num_failure = 0
    
    # generate num_trials random strings and encode and decode them
    for _ in range(num_trials):
        try:
            # generate random message
            msg = generate_binary_array(message_length)
            # encode, simulate error, decode
            encoded = voyager_encode(msg)
            received = simulate_error(encoded, p)
            recovered = voyager_decode(received)

            num_errors = np.sum(recovered != msg)
            num_errors_list.append(num_errors)
        except:
            # if an exception is thrown, meaning the decode completely failed
            # e.g. lambd.formal_derivative()(e) == alpha^(-inf) i.e. additive identity in GF(2^m)
            num_errors_list.append(message_length)
            num_failure += 1

    num_errors_list_list.append(num_errors_list)
    num_failure_list.append(num_failure)

100%|████████████████████████████████████████████████████████████████████████████| 20/20 [1:32:05<00:00, 276.30s/it]


In [74]:
sim_result = np.array(num_errors_list_list)

In [75]:
np.save('sim_result_03.npy', sim_result)

In [76]:
print(num_failure_list)

[1, 0, 0, 0, 0, 3, 0, 1, 1, 3, 1, 0, 0, 0, 2, 3, 2, 2, 1, 0]


In [77]:
np.save('sim_result_failure_03.npy', num_failure_list)

In [54]:
hamming_dist(encoded, received) / 8

37.25

In [56]:
a = RS_255_223.encode(msg)
b = conv.decode(received)

In [61]:
np.sum(convert_to_chunks(a, 8) != convert_to_chunks(b, 8))

23

In [46]:
len(num_errors_list_list)

28